In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import utils
import skutils
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Imputer
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

seed = 47

# Use ggplot style
plt.style.use('ggplot')

%matplotlib inline

/home/stevie/Develop/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')

big = train
big.species = big.species.astype('category')
big.species = big.species.cat.codes

big.species

def transform(data):
    ID = data.id
    X = data.drop(['species', 'id'], axis=1)
    y = data['species']
    return ID, X, y

ID, X, y = transform(big)

In [3]:
def remove_values_attribute(df, attribute, percentage):
    df = df.copy()
    df.loc[df.sample(n=int(percentage * df.shape[0]), random_state=seed).index, attribute] = None
    return df
    
def remove_values(df, percentage):
    np.random.seed(seed)
    df = df.copy()
    for i in df.sample(n=int(percentage * df.shape[0]), random_state=seed).index:
        j = np.random.randint(df.shape[1])
        df.loc[i,df.columns[j]] = None
    return df
    
    
X1 = remove_values_attribute(X, 'margin5', 0.1)
X2 = remove_values_attribute(X, 'margin5', 0.6)
X3 = remove_values_attribute(X, 'shape1', 0.6)
X4 = remove_values_attribute(X, 'shape1', 0.1)

X5 = remove_values(X, 0.1)
X6 = remove_values(X, 0.6)

Xn = {
    'margin5_low': {
        'original': X1
    },
    'margin5_high': {
        'original': X2
    },
    'shape1_low': {
        'original': X3
    },
    'shape1_high': {
        'original': X4
    },
    'all_low': {
        'original': X5
    },
    'all_high':{
        'original': X6
    },
}

In [16]:
def ignore_missing(df):
    return df.dropna(axis=1)

def replace_by_totalmean(df):
    return Imputer(axis=1).fit_transform(df)

def replace_by_classmean(X, y):    
    df = pd.concat([y,X], axis=1)
    
    for col in X.columns[X.isnull().any()].tolist():
        grouped = df.groupby(['species'])[col]
        df = df.merge(grouped.mean().to_frame().reset_index(), left_on='species', right_on='species', how='inner')

        df[col] = df[col + '_x']
        df[col] = df[col].fillna(df[col + '_y'])
        df = df.drop([col + '_x', col + '_y'], axis=1)
    
    return df.drop('species', axis=1)

In [19]:
for k in Xn.keys():
    Xi = Xn[k]
    Xi['ignore'] = ignore_missing(Xi['original'])
    Xi['totalmean'] = replace_by_totalmean(Xi['original'])
    Xi['classmean'] = replace_by_classmean(Xi['original'], y)